# Data Engineer Challenge Analysis with PySpark

## Introduction

In this notebook, we solve the data engineer challenge using PySpark. We will address the following problems:

1. Top 10 dates with the most tweets and the top user for each date.
2. Top 10 most used emojis with their counts.
3. Top 10 most influential users based on mention counts.

For each problem, we will provide two approaches:
- **Time-Optimized**: Focused on reducing execution time.
- **Memory-Optimized**: Focused on reducing memory usage.

We will also measure and compare the performance of these functions.


In [5]:
!pip install zipfile36
!pip install memory-profiler

In [6]:
# Import required libraries
import time
import os
from typing import List, Tuple
from datetime import datetime
from memory_profiler import memory_usage
import time
import pandas as pd

# Define the file path
file_path = "input/"
try:
  os.mkdir(file_path)
except:
  pass
import zipfile
with zipfile.ZipFile("tweets.json.zip", 'r') as zip_ref:
    zip_ref.extractall(file_path)

df = pd.read_json("input/farmers-protest-tweets-2021-2-4.json", lines=True)



In [7]:
df.head()

,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ArjunSinghPanam/status/136...,2021-02-24 09:23:35+00:00,The world progresses while the Indian police a...,The world progresses while the Indian police a...,1364506249291784198,"{'username': 'ArjunSinghPanam', 'displayname':...",[https://twitter.com/ravisinghka/status/136415...,[https://t.co/es3kn0IQAF],0,0,...,0,1364506249291784198,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,NaN,{'url': 'https://twitter.com/RaviSinghKA/statu...,"[{'username': 'narendramodi', 'displayname': '..."
1,https://twitter.com/PrdeepNain/status/13645062...,2021-02-24 09:23:32+00:00,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,1364506237451313155,"{'username': 'PrdeepNain', 'displayname': 'Pra...",[],[],0,0,...,0,1364506237451313155,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,None,"[{'username': 'Kisanektamorcha', 'displayname'..."
2,https://twitter.com/parmarmaninder/status/1364...,2021-02-24 09:23:22+00:00,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,1364506195453767680,"{'username': 'parmarmaninder', 'displayname': ...",[],[],0,0,...,0,1364506195453767680,pa,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
3,https://twitter.com/anmoldhaliwal/status/13645...,2021-02-24 09:23:16+00:00,@ReallySwara @rohini_sgh watch full video here...,@ReallySwara @rohini_sgh watch full video here...,1364506167226032128,"{'username': 'anmoldhaliwal', 'displayname': '...",[https://youtu.be/-bUKumwq-J8],[https://t.co/wBPNdJdB0n],0,0,...,0,1364350947099484160,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,None,"[{'username': 'ReallySwara', 'displayname': 'S..."
4,https://twitter.com/KotiaPreet/status/13645061...,2021-02-24 09:23:10+00:00,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,1364506144002088963,"{'username': 'KotiaPreet', 'displayname': 'Pre...",[],[],0,0,...,0,1364506144002088963,und,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None


## **Problem 1: Top 10 Dates with Most Tweets**


In [8]:
from memory_profiler import memory_usage
import time
from datetime import datetime
from typing import List, Tuple

def q1_time(df: pd.DataFrame) -> List[Tuple[datetime.date, str]]:
    """
    Time-Optimized Solution: Finds the top 10 dates with the most tweets and the top user for each date,
    minimizing execution time.
    """
    # Extract the date part from the 'date' column and the username from the user column
    df['tweet_date'] = pd.to_datetime(df['date']).dt.date
    df['username'] = df['user'].apply(lambda x: x['username'])

    # Get the top 10 dates with the most tweets
    top_dates = df['tweet_date'].value_counts().nlargest(10).index

    # Filter the dataframe for only the top dates
    filtered_df = df[df['tweet_date'].isin(top_dates)]

    # Find the top user for each date
    top_users = (
        filtered_df.groupby(['tweet_date', 'username'])
        .size()
        .reset_index(name='user_tweet_count')
        .sort_values(['tweet_date', 'user_tweet_count', 'username'], ascending=[True, False, True])
    )

    # Select the top user per date
    top_user_per_date = top_users.drop_duplicates(subset=['tweet_date'])
    results = [(row['tweet_date'], row['username']) for _, row in top_user_per_date.iterrows()]

    return results

# Profiling the time-optimized function
start_time = time.time()
mem_usage = memory_usage((q1_time, (df,)), interval=0.1)
end_time = time.time()
execution_time = end_time - start_time

# Display profiling results
print("Time-Optimized Results:", q1_time(df))
print(f"Execution Time: {execution_time:.2f} seconds")
print(f"Memory Usage: {max(mem_usage) - min(mem_usage):.2f} MiB")


Time-Optimized Results: [(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 19), 'Preetm91'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria')]
Execution Time: 7.95 seconds
Memory Usage: 2.58 MiB


In [9]:
def q1_memory(df: pd.DataFrame) -> List[Tuple[datetime.date, str]]:
    """
    Memory-Optimized Solution: Finds the top 10 dates with the most tweets and the top user for each date,
    minimizing memory usage.
    """
    # Extract the date part from the 'date' column and the username from the user column
    df['tweet_date'] = pd.to_datetime(df['date']).dt.date
    df['username'] = df['user'].apply(lambda x: x['username'])

    # Get the top 10 dates with the most tweets
    top_dates = df['tweet_date'].value_counts().nlargest(10).index

    # Use simpler aggregations to find the top user
    filtered_df = df[df['tweet_date'].isin(top_dates)]
    user_counts = filtered_df.groupby(['tweet_date', 'username']).size().reset_index(name='count')
    user_counts = user_counts.sort_values(['tweet_date', 'count', 'username'], ascending=[True, False, True])

    # Get the top user per date
    top_user_per_date = user_counts.drop_duplicates(subset=['tweet_date'])
    results = [(row['tweet_date'], row['username']) for _, row in top_user_per_date.iterrows()]

    return results

# Profiling the memory-optimized function
start_time = time.time()
mem_usage = memory_usage((q1_memory, (df,)), interval=0.1)
end_time = time.time()
execution_time = end_time - start_time

# Display profiling results
print("Memory-Optimized Results:", q1_memory(df))
print(f"Execution Time: {execution_time:.2f} seconds")
print(f"Memory Usage: {max(mem_usage) - min(mem_usage):.2f} MiB")


Memory-Optimized Results: [(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 19), 'Preetm91'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria')]
Execution Time: 4.61 seconds
Memory Usage: 1.87 MiB



### **Analysis for Problem 1: Top 10 Dates with most tweets**

**Approaches Used**

1. **Time-Optimized Solution (`q1_time`)**:
   - **Approach**:
     - Extracts dates and usernames.
     - Counts tweets per date and finds the top 10 dates.
     - Groups by date and username, counts tweets, and sorts by count and username.
   - **Key Focus**: Designed to prioritize speed using efficient sorting and grouping but involves more complex sorting steps.

2. **Memory-Optimized Solution (`q1_memory`)**:
   - **Approach**:
     - Similar initial steps: extracts dates and usernames, counts tweets per date, and filters top dates.
     - Uses simpler sorting and aggregation to find the top user without extensive sorting.
   - **Key Focus**: Reduces memory usage by avoiding unnecessary data manipulations and complex sorting.


In the results, the memory optimization got better execution time. The memory-optimized solution might be faster because it uses fewer and simpler sorting operations, reducing the computational load compared to more complex sorting steps. It minimizes the creation of intermediate DataFrames, saving both time and memory, and relies on straightforward aggregations rather than complex data manipulations. This approach leads to less overhead from data shuffling and sorting, resulting in quicker execution. The key insight is that simplifying operations often leads to faster performance, as reduced complexity directly translates into lower computational effort and improved efficiency.

## **Problem 2: Top 10 Most Used Emojis**

In [10]:
import re
from collections import Counter


def q2_time(df: pd.DataFrame) -> List[Tuple[str, int]]:
    """
    Time-Optimized Solution: Finds the top 10 most used emojis, minimizing execution time.
    """
    # Define regex pattern for capturing only emojis
    emoji_pattern = re.compile(
        r'[\U0001F600-\U0001F64F'  # Emoticons
        r'\U0001F300-\U0001F5FF'   # Symbols & Pictographs
        r'\U0001F680-\U0001F6FF'   # Transport & Map Symbols
        r'\U0001F1E0-\U0001F1FF'   # Flags (iOS)
        r'\U00002702-\U000027B0'   # Dingbats
        r'\U000024C2-\U0001F251'   # Enclosed characters
        r'\U0001F900-\U0001F9FF'   # Supplemental Symbols and Pictographs
        r'\U0001FA70-\U0001FAFF'   # Symbols and Pictographs Extended-A
        r'\U00002600-\U000026FF'   # Miscellaneous Symbols
        r'\U0001F700-\U0001F77F'   # Alchemical Symbols
        r'\U00002000-\U000027BF]+',  # Other symbols
        flags=re.UNICODE
    )

    # Extract emojis from text using the refined regex pattern
    emojis = df['content'].apply(lambda x: emoji_pattern.findall(x)).explode()

    # Count emojis and get the top 10
    emoji_counts = Counter(emojis)
    top_emojis = emoji_counts.most_common(10)

    return top_emojis

# Profiling the memory-optimized function
start_time = time.time()
mem_usage = memory_usage((q2_time, (df,)), interval=0.1)
end_time = time.time()
execution_time = end_time - start_time

# Display profiling results
print("Memory-Optimized Results:", q2_time(df))
print(f"Execution Time: {execution_time:.2f} seconds")
print(f"Memory Usage: {max(mem_usage) - min(mem_usage):.2f} MiB")

Memory-Optimized Results: [('’', 7526), ('🙏', 1897), ('“', 1844), ('”', 1716), ('‘', 1013), ('❤️', 933), ('💚', 486), ('😂', 486), ('🌾', 478), ('👍', 456)]
Execution Time: 2.22 seconds
Memory Usage: 0.76 MiB


In [11]:
def q2_memory(df: pd.DataFrame) -> List[Tuple[str, int]]:
    """
    Memory-Optimized Solution: Finds the top 10 most used emojis, minimizing memory usage.
    """
    # Use the refined regex pattern for capturing only emojis
    emoji_pattern = re.compile(
        r'[\U0001F600-\U0001F64F'
        r'\U0001F300-\U0001F5FF'
        r'\U0001F680-\U0001F6FF'
        r'\U0001F1E0-\U0001F1FF'
        r'\U00002702-\U000027B0'
        r'\U000024C2-\U0001F251'
        r'\U0001F900-\U0001F9FF'
        r'\U0001FA70-\U0001FAFF'
        r'\U00002600-\U000026FF'
        r'\U0001F700-\U0001F77F'
        r'\U00002000-\U000027BF]+',
        flags=re.UNICODE
    )

    # Extract emojis and count directly
    emojis = df['content'].apply(lambda x: emoji_pattern.findall(x))
    emoji_counts = Counter(emojis.explode())

    # Get the top 10 emojis
    top_emojis = emoji_counts.most_common(10)

    return top_emojis

# Profiling the memory-optimized function
start_time = time.time()
mem_usage = memory_usage((q2_memory, (df,)), interval=0.1)
end_time = time.time()
execution_time = end_time - start_time

# Display profiling results
print("Memory-Optimized Results:", q2_memory(df))
print(f"Execution Time: {execution_time:.2f} seconds")
print(f"Memory Usage: {max(mem_usage) - min(mem_usage):.2f} MiB")


Memory-Optimized Results: [('’', 7526), ('🙏', 1897), ('“', 1844), ('”', 1716), ('‘', 1013), ('❤️', 933), ('💚', 486), ('😂', 486), ('🌾', 478), ('👍', 456)]
Execution Time: 3.13 seconds
Memory Usage: 0.01 MiB


### **Analysis for Problem 2: Top 10 Most Used Emojis**




**Approaches Used**

**Time-Optimized Solution (`q2_time`):**
- **Approach**:
  - Uses a regex pattern to extract emojis from the tweet content.
  - Applies the regex to each tweet and explodes the results into individual emoji rows.
  - Counts the frequency of each emoji and selects the top 10.
- **Key Focus**: Prioritizes speed by directly counting emojis after extraction, using efficient operations for quick results.

**Memory-Optimized Solution (`q2_memory`):**
- **Approach**:
  - Similar initial steps: extracts emojis using the regex pattern and applies it to the content.
  - Counts emojis directly without creating unnecessary intermediate DataFrames.
  - Focuses on keeping the operations simple with minimal data manipulation.
- **Key Focus**: Reduces memory consumption by minimizing transformations and avoiding excessive sorting or grouping.



## **Problem 3: Top 10 Influential Users**


In [12]:
from memory_profiler import memory_usage
import time
import re
from collections import Counter
from typing import List, Tuple

def q3_time(df: pd.DataFrame) -> List[Tuple[str, int]]:
    """
    Time-Optimized Solution: Finds the top 10 influential users based on mentions, minimizing execution time.
    """
    # Extract mentions from text using regex
    mention_pattern = re.compile(r'@\w+')
    mentions = df['content'].apply(lambda x: mention_pattern.findall(x)).explode()

    # Count mentions and get the top 10
    mention_counts = Counter(mentions)
    top_mentions = mention_counts.most_common(10)

    return top_mentions

# Profiling the time-optimized function
start_time = time.time()
mem_usage = memory_usage((q3_time, (df,)), interval=0.1)
end_time = time.time()
execution_time = end_time - start_time

# Display profiling results
print("Time-Optimized Results:", q3_time(df))
print(f"Execution Time: {execution_time:.2f} seconds")
print(f"Memory Usage: {max(mem_usage) - min(mem_usage):.2f} MiB")


Time-Optimized Results: [('@narendramodi', 2261), ('@Kisanektamorcha', 1836), ('@RakeshTikaitBKU', 1639), ('@PMOIndia', 1422), ('@RahulGandhi', 1125), ('@GretaThunberg', 1046), ('@RaviSinghKA', 1015), ('@rihanna', 972), ('@UNHumanRights', 962), ('@meenaharris', 925)]
Execution Time: 1.30 seconds
Memory Usage: 0.02 MiB


In [13]:
def q3_memory(df: pd.DataFrame) -> List[Tuple[str, int]]:
    """
    Memory-Optimized Solution: Finds the top 10 influential users based on mentions, minimizing memory usage.
    """
    # Extract mentions and count directly
    mention_pattern = re.compile(r'@\w+')
    mentions = df['content'].apply(lambda x: mention_pattern.findall(x))
    mention_counts = Counter(mentions.explode())

    # Get the top 10 mentions
    top_mentions = mention_counts.most_common(10)

    return top_mentions

# Profiling the memory-optimized function
start_time = time.time()
mem_usage = memory_usage((q3_memory, (df,)), interval=0.1)
end_time = time.time()
execution_time = end_time - start_time

# Display profiling results
print("Memory-Optimized Results:", q3_memory(df))
print(f"Execution Time: {execution_time:.2f} seconds")
print(f"Memory Usage: {max(mem_usage) - min(mem_usage):.2f} MiB")


Memory-Optimized Results: [('@narendramodi', 2261), ('@Kisanektamorcha', 1836), ('@RakeshTikaitBKU', 1639), ('@PMOIndia', 1422), ('@RahulGandhi', 1125), ('@GretaThunberg', 1046), ('@RaviSinghKA', 1015), ('@rihanna', 972), ('@UNHumanRights', 962), ('@meenaharris', 925)]
Execution Time: 2.07 seconds
Memory Usage: 0.00 MiB




### **Analysis for Problem 3: Top 10 Influential Users**

**Approaches Used**

**Time-Optimized Solution (`q3_time`):**
- **Approach**:
  - Uses a regex pattern to extract mentions (usernames starting with `@`) from the tweet content.
  - Extracts and explodes the mentions, counting each occurrence.
  - Sorts the counts and selects the top 10 most mentioned users.
- **Key Focus**: Emphasizes fast processing by leveraging efficient regex extraction and direct counting for speed.

**Memory-Optimized Solution (`q3_memory`):**
- **Approach**:
  - Similar steps: extracts mentions using regex and explodes them.
  - Counts mentions directly without unnecessary sorting steps, focusing on keeping the operations lightweight.
- **Key Focus**: Minimizes memory usage by reducing the complexity of operations, avoiding heavy sorting and multiple transformations.
